In [34]:
import pandas as pd
from datetime import datetime
import time
from random import randint

# Scraping
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re

In [46]:
non_loc_df = pd.read_csv('no_location_tweets.csv', encoding='unicode_escape') #df1
non_loc_df = non_loc_df.drop_duplicates()
#FROM: https://console.apify.com/actors/wHMoznVs94gOcxcZl/runs/5pKhwFdWMrGdadH4I#output
"""df_apify  = pd.read_csv('apify_climate_tweets.csv', encoding='unicode_escape')
df_apify['Time'] = pd.to_datetime(df_apify['Time']).dt.strftime("%Y-%m-%d %H:%M:%S")
df_apify['UserName'] = df_apify['UserName'].str.replace('@', '')
df_apify = df_apify.dropna(subset=['Text'])
df_apify['UserURL'] = "https://twitter.com/" + df_apify['UserName']

non_loc_df = pd.concat([df1, df_apify])
"""
non_loc_df

,Name,UserName,Time,Text,Likes,Retweets,TweetURL,UserURL,Location
0,Adam Gee,DwellerNorwich,3/5/2022 22:44,Replying to \n@Nigel_Farage\nClimate Change a ...,NaN,39.0,https://twitter.com/DwellerNorwich/status/1500...,https://twitter.com/DwellerNorwich,NaN
1,ramona quazzola,r_quazzy,3/5/2022 23:00,Replying to \n@cathywilcox1\nIs anthropologica...,4.0,23.0,https://twitter.com/r_quazzy/status/1500244998...,https://twitter.com/r_quazzy,NaN
2,Sam Murik,MurikSam,3/5/2022 21:31,"Replying to \n@nathaliejacoby1\nMan, greed, po...",1.0,40.0,https://twitter.com/MurikSam/status/1500222512...,https://twitter.com/MurikSam,NaN
3,TrapGod.J,jacqueswalker12,3/5/2022 23:25,Replying to \n@marcorubio\nClimate change will...,NaN,45.0,https://twitter.com/jacqueswalker12/status/150...,https://twitter.com/jacqueswalker12,NaN
4,Animal (*Farm),grottyscotty,3/5/2022 22:54,Replying to \n@ProfPCDoherty\nThe great sadnes...,3.0,16.0,https://twitter.com/grottyscotty/status/150024...,https://twitter.com/grottyscotty,NaN
...,...,...,...,...,...,...,...,...,...
4260,Des Moines Brewster,DSMbrewster,3/5/2022 23:46,Currently vacuuming gallons and gallons of wa...,1.0,21.0,https://twitter.com/DSMbrewster/status/1500256...,https://twitter.com/DSMbrewster,NaN
4261,Polly Hemming,pollyjhemming,3/5/2022 23:35,More concerning than the announced change to E...,57.0,86.0,https://twitter.com/pollyjhemming/status/15002...,https://twitter.com/pollyjhemming,NaN
4262,Suzanne Mrozak,smrozak,3/5/2022 22:12,"[VIDEO] MGH Center for Climate Change, Climate...",2.0,2.0,https://twitter.com/smrozak/status/15002327474...,https://twitter.com/smrozak,NaN
4263,ÃÂÃÂÃÂÃÂ°ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃ...,SevNightingale,3/5/2022 22:31,We have to act. We have to reverse climate cha...,3.0,13.0,https://twitter.com/SevNightingale/status/1500...,https://twitter.com/SevNightingale,NaN


In [48]:
## Scrape Twitter to get Location
def get_tweet_info(user_url, driver):
    try:
        driver.get(user_url)
    except:
        time.sleep(1)
        try:
            driver.get(user_url)
        except:
            print('Driver get url failed', user_url)
            location = None

    # Wait for the page to load, adjust the timeout as needed
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid="tweet"]')))
    except:
        time.sleep(1)

    resp = driver.page_source
    soup=BeautifulSoup(resp,'html.parser')

    
    # Find the element with the specific class name
    location_element = soup.find_all('span', class_='css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0', style='text-overflow: unset;')


    # Regular expression pattern to extract 'Colorado, USA'
    location_pattern = re.compile(r'<span class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0" style="text-overflow: unset;">\s*<span class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0" style="text-overflow: unset;">(.*?)</span>\s*</span>')

    # Find all matches in the text content of each element
    matches = []
    for element in location_element:
        try:
            loc = location_pattern.search(str(element)).group(1).strip()
        except:
            loc = 'ERROR'

        if  loc!='ERROR' and len(loc)<= 200:
            matches.append(loc)
    
    if len(matches)>0:
        location = matches[0]
    else:
        location = 'ERROR'
    
    return location

path = Service("chromedriver.exe")

# set up the webdriver
driver = webdriver.Chrome(service=path)


locations = {}
for user_url in non_loc_df['UserURL']:
    if user_url not in locations:  # Check if location is not already found
        location = get_tweet_info(user_url, driver)
        locations[user_url] = location
        time.sleep(randint(1, 3))

# Create a new column in non_loc_df using the locations dictionary
non_loc_df['Location'] = non_loc_df['UserURL'].map(locations.get)

    
driver.close()

Driver get url failed nan


In [49]:
non_loc_df.to_csv('Tweets_with_loc.csv')